In [1]:
import pickle
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
def readData(datafile):
    datafileImport = "data/" + datafile
    infile = open(datafileImport, 'rb')
    new_dict = pickle.load(infile)
    return new_dict

In [3]:
data=["TY_Koc_Sales.pkl","TY_Koc_Basket.pkl","TY_Koc_Fav.pkl","TY_Koc_Gender.pkl","TY_Koc_Impression.pkl"\
      ,"TY_Koc_Price.pkl","TY_Koc_Quantity_Demand.pkl","TY_Koc_Quantity.pkl","TY_Koc_Rating_Review.pkl"\
      ,"TY_Koc_Removefromfav.pkl","TY_Koc_Size_Atts.pkl","TY_Koc_Visit.pkl"]

In [4]:
datas={"salesData":[],
       "basket":[],
       "fav":[],
       "gender":[],
       "impression":[],
       "price":[],
       "demand":[],
       "quantity":[],
       "rating":[],
       "removeFromFav":[],
       "sizeAtt":[],
       "visit":[]
       }

In [5]:
keys=list(datas.keys())
for i in range(len(data)):
    datas[keys[i]]=readData(data[i])

In [6]:
min_date = min(datas["salesData"]["order_date"])
max_date = max(datas["salesData"]["order_date"])
print("Avaliable min date = ", min_date, "\nAvaliable max date = ", max_date)
print("Total timeframe of the salesData",\
      (max(datas["salesData"]["order_date"])-min(datas["salesData"]["order_date"])).days+1)

Avaliable min date =  2021-04-01 
Avaliable max date =  2021-10-24
Total timeframe of the salesData 207


In [7]:
# Zero padding
sales= datas["salesData"].sort_values(by=['product_id','order_date'])
basket= datas["basket"].sort_values(by=['product_id','date'])
visit= datas["visit"].sort_values(by=['product_id','date'])
impression= datas["impression"].sort_values(by=['product_id','date'])
demand= datas["demand"].sort_values(by=['product_id','date'])
removeFromFav= datas["removeFromFav"].sort_values(by=['product_id','date'])

# Elongate
fav= datas["fav"].sort_values(by=['product_id','date'])
rating= datas["rating"].sort_values(by=['product_id','date'])
#Needs to be analyzed
price= datas["price"].sort_values(by=['product_id','created_date'])


# For clustering, no place in regression
attr = datas["sizeAtt"]
gender= datas["gender"]

# Discarded
stock= datas["quantity"].sort_values(by=['product_id','date'])

In [62]:
price.head(50)

,product_id,price,created_date
123931,17629671,5.99,2021-04-01
1788506,4229743431,0.00,2021-05-04
2247524,27176044684,29.90,2021-06-21
828648,27176044684,52.90,2021-06-28
2457587,27176044684,59.90,2021-08-14
2259949,29503436423,370.11,2021-09-05
495439,29503436423,473.14,2021-09-15
2275900,29503436423,433.22,2021-10-11
1100422,44660371726,24.99,2021-10-19
1679606,52106518381,25.76,2021-10-15


In [70]:
price

,product_id,price,created_date
123931,17629671,5.99,2021-04-01
1788506,4229743431,0.00,2021-05-04
2247524,27176044684,29.90,2021-06-21
828648,27176044684,52.90,2021-06-28
2457587,27176044684,59.90,2021-08-14
...,...,...,...
1385144,4999970767511713,43.54,2021-05-02
1390626,4999970767511713,38.21,2021-05-16
1612359,4999970767511713,39.90,2021-05-30
2360389,4999970767511713,0.00,2021-06-09


In [78]:
list(grouped_prices)

[(4229743431,
           product_id  price created_date
  1788506  4229743431    0.0   2021-05-04),
 (68711544940,
           product_id  price created_date
  400824  68711544940    0.0   2021-08-20),
 (72641991820,
            product_id  price created_date
  1138857  72641991820    0.0   2021-04-12
  1858814  72641991820    0.0   2021-06-25),
 (109386384355,
             product_id  price created_date
  2400424  109386384355    0.0   2021-07-02),
 (153703556518,
            product_id  price created_date
  964225  153703556518    0.0   2021-08-18),
 (167212238858,
             product_id  price created_date
  2340489  167212238858    0.0   2021-05-29),
 (172817120004,
             product_id  price created_date
  2317958  172817120004    0.0   2021-09-30),
 (175036287311,
            product_id  price created_date
  262636  175036287311    0.0   2021-10-06),
 (193388911983,
             product_id  price created_date
  1057091  193388911983    0.0   2021-04-27),
 (224034861766,
     

In [95]:
count = 0
fail_cases = []
zero_price = price[price.price == 0].drop_duplicates(subset=['product_id'])
zero_price_id = list(zero_price.product_id)
price_mask = price.product_id.isin(zero_price_id)
#grouped_prices = zero_price.groupby("product_id")  
new_price = price[price_mask]

sales_mask = sales.product_id.isin(zero_price_id)
new_sales = sales[sales_mask]

In [98]:
new_price.head(50)

,product_id,price,created_date
1788506,4229743431,0.00,2021-05-04
400824,68711544940,0.00,2021-08-20
1138857,72641991820,0.00,2021-04-12
2095007,72641991820,71.33,2021-04-19
92970,72641991820,73.33,2021-05-22
1858814,72641991820,0.00,2021-06-25
2400424,109386384355,0.00,2021-07-02
964225,153703556518,0.00,2021-08-18
1793494,167212238858,129.99,2021-05-04
2340489,167212238858,0.00,2021-05-29


In [ ]:
for id in zero_price_id:
    sale = new_sales[new_sales.product_id == id]
    if(len(sale)>0):
        fail_cases.append(id)
        print(id)

In [111]:
fail_cases

[579105381954,
 579105381954,
 1029365666706,
 1056670444899,
 1096394326057,
 1183451511624,
 1859927485064,
 1873611551028,
 2156659498487,
 2337856421707,
 3044293521541,
 3110009578558,
 3422928547310,
 3555311379167,
 3701497572874,
 4095154352402,
 4188174528786,
 4314922423265,
 4453523056544,
 4643759585034,
 4848765975047,
 5563500160976,
 5582065339048,
 5686394019820,
 5790531683911,
 5974926527083,
 6040180737293,
 6904567755612,
 7024824575958,
 7498074120386,
 7574125588313,
 7593267464777,
 8536091024162,
 8616847869415,
 8670549097007,
 8733536381559,
 8741459616992,
 8748203781578,
 9599330546223,
 9600402901208,
 9900080159502,
 10903190481635,
 10916112521533,
 11067744932573,
 11340121756401,
 11442227113057,
 11600185966071,
 11609585504477,
 12295770974071,
 12313048317390,
 12333025041328,
 12427052447853,
 12756176523191,
 12850267081740,
 13041332512419,
 13119647472399,
 13142506281048,
 14079976029018,
 14322010914751,
 14448490895733,
 14595527120460,
 14693

In [113]:
new_sales[new_sales.product_id == 4188174528786]

,product_id,brand_id,order_date,sales,current_bu_group_name,current_category_name
36856,4188174528786,5800877208,2021-05-11,1,Ev,Kupalar
40693,4188174528786,5800877208,2021-05-27,1,Ev,Kupalar
480029,4188174528786,5800877208,2021-06-11,1,Ev,Kupalar


In [114]:
new_price[new_price.product_id == 4188174528786]

,product_id,price,created_date
223686,4188174528786,39.80,2021-04-15
1789038,4188174528786,50.00,2021-05-04
479899,4188174528786,39.99,2021-06-06
1737920,4188174528786,39.90,2021-06-10
1086193,4188174528786,32.80,2021-06-16
2241767,4188174528786,0.00,2021-06-21
1863670,4188174528786,39.75,2021-06-25
157136,4188174528786,38.75,2021-10-07


In [8]:
stock[stock.product_id == 4188174528786]

,product_id,date,quantity


In [ ]:
for id in zero_price_id:
    prices = new_price(new_price.product_id == id)
    if(len(prices) == 1 and (0 in prices)):
        count += 1
        fail_cases.append(id)
        print("Test failed: sale while zero price")
        break

In [25]:
print("Number of uniqe products in each table")
for name,d in datas.items():
    print(name,len(Counter(d["product_id"]).keys())) # equals to list(set(words)))

#len(Counter(datas["salesData"]["product_id"]).values()) # counts the elements' frequency

Number of uniqe products in each table
salesData 68310
basket 126724
fav 145346
gender 327774
impression 78716
price 718986
demand 16275
quantity 1691
rating 32606
removeFromFav 127377
sizeAtt 959317
visit 310131


In [18]:
#Stock data for a specific product
stock = datas["quantity"]
stock[(stock.product_id == 2056551686182716)].sort_values(by=['date'])

,product_id,date,quantity
86846,2056551686182716,2021-04-01,29
70161,2056551686182716,2021-04-02,26
96608,2056551686182716,2021-04-03,22
94748,2056551686182716,2021-04-04,21
111154,2056551686182716,2021-04-05,20
...,...,...,...
20308,2056551686182716,2021-10-12,5
31971,2056551686182716,2021-10-13,1
0,2056551686182716,2021-10-14,0
82307,2056551686182716,2021-10-21,0


In [52]:
datas["salesData"][(datas["salesData"].product_id == 3557874004865180)].sort_values(by=['order_date'])

,product_id,brand_id,order_date,sales,current_bu_group_name,current_category_name
442376,3557874004865180,940277741,2021-05-03,1,Ev,Avize


In [34]:
datas["demand"][(datas["demand"].product_id == 2056551686182716)].sort_values(by=['date'])

,product_id,date,quantity_demand
95954,2056551686182716,2021-04-05,1
28433,2056551686182716,2021-04-11,1
160,2056551686182716,2021-04-21,1
46070,2056551686182716,2021-04-23,1
41440,2056551686182716,2021-04-24,1
31101,2056551686182716,2021-04-25,1
88916,2056551686182716,2021-04-26,1
78095,2056551686182716,2021-04-27,2
67007,2056551686182716,2021-04-29,1
56418,2056551686182716,2021-05-03,1


In [45]:
max(datas["salesData"]["order_date"])-min(datas["salesData"]["order_date"])
min(datas["fav"].sort_values(by=['date']).fav)

1

In [53]:
datas["impression"][datas["impression"].product_id == 3557874004865180].sort_values(by=['date']).head(50)

,product_id,date,impression
251292,3557874004865180,2021-04-01,10
156218,3557874004865180,2021-04-02,5
73154,3557874004865180,2021-04-03,14
801739,3557874004865180,2021-04-04,11
745268,3557874004865180,2021-04-05,1
434723,3557874004865180,2021-04-08,1
1259058,3557874004865180,2021-04-09,2
1618981,3557874004865180,2021-04-10,7
1586574,3557874004865180,2021-04-11,9
1485438,3557874004865180,2021-04-12,5


In [71]:
len(Counter(datas["price"][datas["price"].price == 0].sort_values(by=['created_date']).product_id).keys())


188743

In [70]:
datas["price"][datas["price"].price == 0].sort_values(by=['created_date'])

,product_id,price,created_date
107105,1622646028236011,0.0,2021-04-01
107412,4517481808258390,0.0,2021-04-01
107411,2952563082286010,0.0,2021-04-01
107410,1471962003079782,0.0,2021-04-01
107409,4800161062261114,0.0,2021-04-01
...,...,...,...
2429998,1077072651441408,0.0,2021-10-24
2429997,1229835078177613,0.0,2021-10-24
2429996,47595181075257,0.0,2021-10-24
2430002,3023489139003682,0.0,2021-10-24


In [57]:
datas["price"]

,product_id,price,created_date
0,4132725922387191,0.00,2021-07-25
1,1854209447749994,0.00,2021-07-25
2,3385201861738318,0.00,2021-07-25
3,4901072044437340,0.00,2021-07-25
4,4981538539553204,0.00,2021-07-25
...,...,...,...
2486350,2803183558627727,119.84,2021-04-07
2486351,2501867247806930,1214.69,2021-04-07
2486352,729286085459627,2813.38,2021-04-07
2486353,4995275069845093,247.93,2021-04-07


In [64]:
datas["salesData"][(datas["salesData"].product_id == 4132725922387191)].sort_values(by=['order_date'])

,product_id,brand_id,order_date,sales,current_bu_group_name,current_category_name


In [23]:
#dates don't seperate from elongate function, these are necessary
dates = sorted(datas["salesData"]["order_date"].unique())
min_date = min(datas["salesData"]["order_date"])
max_date = max(datas["salesData"]["order_date"])
# Example: new_data_ls = elongate(fav.)
def elongate(df,pid,col_name, date_name="date"):
    def get_len(first,last):
        return (last-first).days
        
    
    def create_list(val, first, last):
        return [val for i in range(get_len(first,last))]
    
    p_df = df[df.product_id == pid]
    f = list(p_df[col_name])
    my_date = list(p_df[date_name])
    
    
    #conditions
    f_len = len(f)
    d_len = len(my_date)
    
    if d_len != f_len : raise Exception("Len mismatch")
    if f_len == 0 : return [0 for i in range(len(dates))]
    init_val = f[0]
    if f_len == 1 : return [init_val for i in range(len(dates))]
    #first_part
    ls = create_list(init_val,min_date, my_date[0])
    #middle_part
    first_dates = my_date[:-1]
    second_dates = my_date[1:]
    data = zip(f, first_dates, second_dates)
    
    for d, first, last in data:
        ls.extend(create_list(d, first, last))
    
    #final_part
    final_val = f[-1]
    final_date = my_date[-1]
    
    ls.extend(create_list(final_val, final_date, max_date))
    return ls
        
    
    

In [24]:
elongate(fav ,68413811038, "fav")

[5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 2,
 4,
 9,
 5,
 4,
 10,
 7,
 3,
 5,
 3,
 5,
 4,
 7,
 6,
 6,
 2,
 2,
 5,
 1,
 1,
 3,
 4,
 3,
 4,
 1,
 2,
 2,
 4,
 2,
 4,
 4,
 1,
 3,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 4,
 1,
 1,
 2,
 2,
 2,
 2,
 4,
 1,
 3,
 2,
 1,
 2,
 3,
 1,
 1,
 1,
 1,
 1,
 3,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 4,
 4,
 2,
 2,
 2,
 1,
 4,
 2,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 4,
 2,
 8,
 3,
 4,
 5,
 5,
 5,
 2,
 3,
 1,
 7,
 2,
 1,
 3,
 1,
 3,
 7,
 2,
 1,
 4,
 4,
 2,
 1,
 2,
 2,
 2,
 3,
 2,
 3,
 3,
 2,
 5,
 2,
 1,
 1,
 3,
 3,
 2,
 6,
 3,
 3,
 9,
 9,
 1,
 1,
 2,
 3,
 3,
 3,
 4,
 13,
 3,
 3,
 4,
 6,
 3,
 2,
 4,
 1,
 1,
 7,
 3,
 2,
 7,
 6,
 2,
 7,
 5,
 2,
 1,
 2]

In [26]:
fav[fav.product_id == 68413811038].tail(5)

,product_id,date,fav
1237005,68413811038,2021-10-20,5
2023160,68413811038,2021-10-21,2
1712869,68413811038,2021-10-22,1
513047,68413811038,2021-10-23,2
60970,68413811038,2021-10-24,1


In [17]:
ls = [1,2,3]
ls[1:2] = 0

TypeError: can only assign an iterable

In [19]:
ls = [1,2,3,4,5]
list()

[(1, 2), (2, 3), (3, 4), (4, 5)]

In [20]:
ls[-1]

5

In [9]:
rating.head(50)

,product_id,date,reviewCount,rating
164017,68413811038,2021-04-02,1,5.000000
94993,68413811038,2021-04-10,1,5.000000
145380,68413811038,2021-04-14,1,5.000000
166521,68413811038,2021-04-16,1,5.000000
173983,68413811038,2021-04-17,2,5.000000
53321,68413811038,2021-04-20,3,5.000000
104126,68413811038,2021-04-25,1,4.000000
112565,68413811038,2021-04-27,1,5.000000
102165,68413811038,2021-05-06,1,5.000000
110352,68413811038,2021-05-07,1,5.000000
